In [1]:
import os
os.environ["SYCL_CACHE_PERSISTENT"] = "1"

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain_community.embeddings import IpexLLMBgeEmbeddings



In [2]:
loader=PyPDFLoader('downloaded_documents/gous2.pdf')
paginas = loader.load()

In [3]:
len(paginas)

11

In [4]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=650,
    chunk_overlap=80,
    length_function=len
)

In [5]:
docs=text_splitter.split_documents(paginas)

In [6]:
embedding_model = IpexLLMBgeEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "xpu"},
    encode_kwargs={"normalize_embeddings": True},
    query_instruction="Identify and extract the names of the main countries studied or focused on in this research",
)
vectorstore=Chroma.from_documents(documents=docs, embedding=embedding_model)

c:\Users\renat\miniconda3\envs\llm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\renat\miniconda3\envs\llm\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\renat\miniconda3\envs\llm\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-12 23:50:26,915 - INFO - intel_extension_for_pytorch auto imported
2024-07-12 23:50:26,916 - INFO - Load pretrained Sent

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def ollama_llm(question,context):
    formatted_question = f"Question: {question}\n\nContext (Answer only about the content of the provided context):{context}"
    response =ollama.chat(model='llama3:8b',
                          messages=[{"role": "user", "content": formatted_question}],
                          options={'temperature':0})
    return response['message']['content']

In [8]:
retriever=vectorstore.as_retriever()
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [9]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context=combine_docs(retrieved_docs)
    return ollama_llm(question,formatted_context)

In [10]:
result=rag_chain("In which countries was the research presented in the past texr carried outt ?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-12 23:50:42,813 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [11]:
print(result)

The research presented was carried out in Guinea-Bissau, as mentioned in the text: "a school ‐based survey in Guinea ‐Bissau found that one ‐ third of adolescents in secondary schools used mobile internet daily..." (Gunnlaugsson et al., 2020).


In [6]:

sentence = "IPEX-LLM is a PyTorch library for running LLM on Intel CPU and GPU (e.g., local PC with iGPU, discrete GPU such as Arc, Flex and Max) with very low latency."
query = "What is IPEX-LLM?"

text_embeddings = embedding_model.embed_documents([sentence, query])
print(f"text_embeddings[0][:10]: {text_embeddings[0][:10]}")
print(f"text_embeddings[1][:10]: {text_embeddings[1][:10]}")

query_embedding = embedding_model.embed_query(query)
print(f"query_embedding[:10]: {query_embedding[:10]}")

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


text_embeddings[0][:10]: [0.035797119140625, 0.032745361328125, -0.0166778564453125, 0.007411956787109375, 0.016265869140625, -0.0019683837890625, -0.0028476715087890625, -0.041412353515625, 0.0310211181640625, 0.054412841796875]
text_embeddings[1][:10]: [0.031524658203125, 0.031768798828125, -0.0030384063720703125, 0.004299163818359375, 0.0049896240234375, -0.02679443359375, -0.0058441162109375, -0.022491455078125, 0.0516357421875, 0.059295654296875]


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.91it/s]

query_embedding[:10]: [0.035614013671875, 0.0299530029296875, 0.005218505859375, 0.005504608154296875, 0.0092926025390625, -0.0364990234375, -0.0203094482421875, -0.0355224609375, 0.03375244140625, 0.0633544921875]
